In [ ]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, accuracy_score, mean_squared_error
from matplotlib.pyplot import figure
import scipy as sp
import numpy as np
from tensorflow.keras.models import load_model
from numpy import savetxt
from google.colab import drive 
drive.mount('/content/gdrive')
################################################################################
file_prefix = '/content/gdrive/My Drive/Xc'
file_suffix = '_test.npy'
file_suffix_p = '_testP.npy'
num_files = 21
Xc = {}
Xcp = {}
# Iterate over the range of file indices
for i in range(num_files):
    file_name = file_prefix + str(i) + file_suffix
    file_name_p = file_prefix + str(i) + file_suffix_p
    loaded_array = np.load(file_name)
    loaded_array_p = np.load(file_name)
    # Create a key based on the file index
    key = f'Xc{i}_test'
    key_p = f'Xc{i}_testP'
    # Add the loaded array to the dictionary
    Xc[key] = loaded_array
    Xcp[key_p] = loaded_array_p
# Load Yc_test separately
Yc_test = np.load('/content/gdrive/My Drive/Yc_test.npy')
Yc_testP = np.load('/content/gdrive/My Drive/Yc_testP.npy')
################################################################################
CLSTM_model = load_model('/content/gdrive/My Drive/CNN_LSTM.h5')
LSTM_model = load_model('/content/gdrive/My Drive/CNN_LSTM_MLP.h5')
BLSTM_model = load_model('/content/gdrive/My Drive/CNN_BLSTM_MLP.h5')
Coop_model = load_model('/content/gdrive/My Drive/Cooperative_LSTM.h5')
##################################
S = 4
Sp = 3
key = f'Xc{14}_test'
key_p = f'Xc{14}_testP'
Xc14_test = Xc[key] 
Xc14_testP = Xcp[key_p] 
i = arange(S+1)*int(Xc14_test.shape[0]/S)
p = arange(Sp+1)*int(Xc14_testP.shape[0]/Sp)
Xu_test = Xc14_test[i[0]:i[1]]
Xx_test = Xc14_test[i[1]:i[2]]
Xy_test = Xc14_test[i[2]:i[3]]
Xz_test = Xc14_test[i[3]:i[4]]
###
Xx_testP = Xc14_testP[p[0]:p[1]]
Xy_testP = Xc14_testP[p[1]:p[2]]
Xz_testP = Xc14_testP[p[2]:p[3]]

fp, fp1, fp2, fp3 = dict(), dict(), dict(), dict()
tp, tp1, tp2, tp3 = dict(), dict(), dict(), dict()
auc, auc1, auc2, auc3 = dict(), dict(), dict(), dict()
y_CLSTM = CLSTM_model.predict([Xu_test, Xx_test, Xy_test, Xz_test])
y_LSTM = LSTM_model.predict([Xu_test, Xx_test, Xy_test, Xz_test])
y_BLSTM = BLSTM_model.predict([Xu_test, Xx_test, Xy_test, Xz_test])
y_Coop = Coop_model.predict([Xx_testP, Xy_testP, Xz_testP])
for i in range(2):
    fp[i], tp[i], _ = roc_curve(Yc_test[:, i], y_CLSTM[:, i])
    fp1[i], tp1[i], _ = roc_curve(Yc_test[:, i], y_LSTM[:, i])
    fp2[i], tp2[i], _ = roc_curve(Yc_test[:, i], y_BLSTM[:, i])
    fp3[i], tp3[i], _ = roc_curve(Yc_test[:, i], y_Coop[:, i])
    roc_auc[i] = auc(fp[i], tp[i])
    roc_auc1[i] = auc(fp1[i], tp1[i])
    roc_auc2[i] = auc(fp2[i], tp2[i])
    roc_auc3[i] = auc(fp3[i], tp3[i])
###
figure(figsize=(12, 10))
plt.figure(1)
plt.xscale('log')
plt.plot(fp[1], tp[1],marker='o', linestyle='--', markevery = int(len(fp2[1]) / 20), lw=2, label='CNN_LSTM (AUC = %0.2f )' % roc_auc[1])
plt.plot(fp1[1], tp1[1],marker='D', linestyle='-.', markevery = int(len(fp3[1]) / 20), lw=2, label='CNN_LSTM_MLP (AUC = %0.2f )' % roc_auc1[1])
plt.plot(fp2[1], tp2[1],marker='v', linestyle=':', markevery = int(len(fp4[1]) / 20), lw=2, label='CNN_BLSTM_MLP (AUC = %0.2f )' % roc_auc2[1])
plt.plot(fp3[1], tp3[1], marker='<', linestyle='-', markevery = int(len(fp9) / 20), lw=2, label='Cooperative LSTM [17] (AUC = %0.2f )' % auc3[1])
plt.xlim([0.001, 1])
plt.xlabel('probability of false alarm', fontsize= 20)
plt.ylabel('probability of detection', fontsize= 20)
plt.legend(loc='upper left', fontsize= 16)
plt.grid(True, which="both")
plt.xticks(fontsize=16) 
plt.yticks(fontsize=16) 
plt.show()
################################################################################
def linear_interpolation(x, x1, y1, x2, y2):
    return y1 + ((y2 - y1) / (x2 - x1)) * (x - x1)
def Pd_SNR(pfa, pd):
  desired_pfa = 0.1
  if desired_pfa in pfa:
    desired_pd = pd[np.where(pfa == desired_pfa)]
  else:
    lower_idx = np.where(pfa < desired_pfa)[-1][-1]
    upper_idx = np.where(pfa > desired_pfa)[0][0]
    lower_pfa = pfa[lower_idx]
    upper_pfa = pfa[upper_idx]
    lower_pd = pd[lower_idx]
    upper_pd = pd[upper_idx]
    desired_pd = linear_interpolation(desired_pfa, lower_pfa, lower_pd, upper_pfa, upper_pd)
  return desired_pd
CLSTM, LSTM_, BLSTM, Coop = [], [], [], []
for i in range(num_files, - 1, -1, -1)):
  key = f'Xc{i}_test'
  key_p = f'Xc{i}_testP'
  Xc_test = Xc[key] 
  Xc_testP = Xcp[key_p] 
  Xu_test = Xc_test[i[0]:i[1]]
  Xx_test = Xc_test[i[1]:i[2]]
  Xy_test = Xc_test[i[2]:i[3]]
  Xz_test = Xc_test[i[3]:i[4]]
  Xx_testP = Xc_testP[p[0]:p[1]]
  Xy_testP = Xc_testP[p[1]:p[2]]
  Xz_testP = Xc_testP[p[2]:p[3]]
  y_CLSTM = CLSTM_model.predict([Xu_test, Xx_test, Xy_test, Xz_test])
  y_LSTM = LSTM_model.predict([Xu_test, Xx_test, Xy_test, Xz_test])
  y_BLSTM = BLSTM_model.predict([Xu_test, Xx_test, Xy_test, Xz_test])
  y_Coop = Coop_model.predict([Xx_testP, Xy_testP, Xz_testP])
  fp[1], tp[1], _ = roc_curve(Yc_test[:, 1], y_CLSTM[:, 1])
  fp1[1], tp1[1], _ = roc_curve(Yc_test[:, 1], y_LSTM[:, 1])
  fp2[1], tp2[1], _ = roc_curve(Yc_test[:, 1], y_BLSTM[:, 1])
  fp3[1], tp3[1], _ = roc_curve(Yc_test[:, 1], y_Coop[:, 1])
  CLSTM.append(Pd_SNR(fp[1], tp[1]))
  LSTM_.append(Pd_SNR(fp1[1], tp1[1]))
  BLSTM.append(Pd_SNR(fp2[1], tp2[1]))
  Coop.append(Pd_SNR(fp3[1], tp3[1]))
################################################################################
SNR = np.arange(-20,1,1)
figure(figsize=(12, 10))
plt.figure(1)
plt.plot(SNR, CLSTM,marker='o', linestyle='-', lw=2, label='CNN_LSTM')
plt.plot(SNR, LSTM_,marker='D', linestyle='-', lw=2, label='CNN_LSTM_MLP')
plt.plot(SNR, BLSTM,marker='v', linestyle='-', lw=2, label='CNN_BLSTM_MLP')
plt.plot(SNR, Coop,marker='<', linestyle='-', lw=2, label='Cooperative LSTM')
plt.xlabel('probability of false alarm', fontsize= 20)
plt.ylabel('probability of detection', fontsize= 20)
plt.legend(loc='upper left', fontsize= 16)
plt.grid(True, which="both")
plt.xticks(fontsize=16) 
plt.yticks(fontsize=16) 
plt.show()

1